<span style="font-size: 12pt; font-family: &quot;Times New Roman&quot;, serif; color: rgb(0, 0, 0); background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; white-space-collapse: preserve;">1. In the last homework assignment, you created a list of territories that required training because they did not have at least one month during 2013 with sales of more than $750,000. Working with this sample principle, use a CASE statement to create a list of all territories in alphabetical order, with a new column called "TrainingRequired" that has a value of either "Yes" or "No."</span>

In [55]:
SELECT DISTINCT ST.Name, --pulling the distinct sales territory name
CASE 
    WHEN MAX(SUM(SOH.Subtotal)) OVER (PARTITION BY ST.Name) > 750000 --creating a partition at 750000 and compares the highest value of the monthly subtotal for each region to 750000
        THEN 'No' --no training is needed because the goal was hit
    ELSE 'Yes' --training is needed because the goal was not hit
    END AS TrainingRequired --assigns it as training required 
FROM Sales.SalesOrderHeader SOH --pulling information from Sales Order Header
INNER JOIN Sales.SalesTerritory ST --Inner joining sales territory with SOH
    ON SOH.TerritoryID = ST.TerritoryID --on the territory column for both SOH and ST
WHERE DATEPART(Year, SOH.OrderDate) = 2013 --only selects the year 2013 as the date 
GROUP BY ST.Name, DATEPART(Year, SOH.OrderDate), DATEPART(Month, SOH.OrderDate) --groups by the year, and then the month
ORDER BY ST.Name ASC --ordered by names ascending so that the names are in alphabetical order

(10 rows affected)

Total execution time: 00:00:00.058

Name,TrainingRequired
Australia,Yes
Canada,No
Central,Yes
France,No
Germany,Yes
Northeast,Yes
Northwest,No
Southeast,Yes
Southwest,No
United Kingdom,Yes


<span style="font-size: 12pt; font-family: &quot;Times New Roman&quot;, serif; color: rgb(0, 0, 0); background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; white-space-collapse: preserve;">2. The sales manager wants to know how many sales orders were shipped to each country and state/province.&nbsp; Your result set should contain three columns: Country, StateProvince, and TotalOrders in alphabetical order by country, then showing the greatest number of TotalOrders.</span>

<span style="font-size: 12pt; font-family: &quot;Times New Roman&quot;, serif; color: rgb(0, 0, 0); background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; text-decoration-line: underline; text-decoration-skip-ink: none; vertical-align: baseline; white-space-collapse: preserve;">Part A</span>

<span style="font-size: 12pt; font-family: &quot;Times New Roman&quot;, serif; color: rgb(0, 0, 0); background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; white-space-collapse: preserve;">Write a single query that shows the total number of sales orders from each country and each state/province within each country. Use the full name of each country and state/province.</span>

In [46]:
SELECT CR.Name AS Country_Name --begin by grabbing the country name and renaming it
        ,SP.Name AS State_Province_Name --then the state/province name
        ,COUNT(SOH.SalesOrderID) AS Orders --and using SalesOrderID to count the number of orders
FROM Sales.SalesOrderHeader SOH --calling the SalesOrder Header so I can get the orders info
JOIN Person.Address PA --but then joining it with the person addresses
    ON SOH.ShipToAddressID = PA.AddressID --so I can link to the province table
JOIN Person.StateProvince SP --which I do here, so I can get state/province names
    ON PA.StateProvinceID = SP.StateProvinceID --and link by the province ID
JOIN Person.CountryRegion CR --and then add in the country region table so I can get country names
    ON SP.CountryRegionCode = CR.CountryRegionCode --and link on the country code
GROUP BY CR.Name, SP.Name --then group by Country name, then province name
ORDER BY CR.Name ASC, COUNT(SOH.SalesOrderID) DESC --and then sort by ascending names, descending count

(70 rows affected)

Total execution time: 00:00:00.156

Country_Name,State_Province_Name,Orders
Australia,New South Wales,3009
Australia,Victoria,1710
Australia,Queensland,1461
Australia,South Australia,466
Australia,Tasmania,197
Canada,British Columbia,3472
Canada,Ontario,339
Canada,Quebec,146
Canada,Alberta,90
Canada,Manitoba,12


<span style="font-size: 12pt; font-family: &quot;Times New Roman&quot;, serif; color: rgb(0, 0, 0); background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; text-decoration-line: underline; text-decoration-skip-ink: none; vertical-align: baseline; white-space-collapse: preserve;">Part B</span>

<span style="font-size: 12pt; font-family: &quot;Times New Roman&quot;, serif; color: rgb(0, 0, 0); background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; white-space-collapse: preserve;">Modify your query to only show cases with 100 or more orders, sorted in the same fashion as Part A.</span>

In [45]:
--All I have to do for this one is add in a having clause so my comments are all the same
SELECT CR.Name AS Country_Name --begin by grabbing the country name and renaming it
        ,SP.Name AS State_Province_Name --then the state/province name
        ,COUNT(SOH.SalesOrderID) AS Orders --and using SalesOrderID to count the number of orders
FROM Sales.SalesOrderHeader SOH --calling the SalesOrder Header so I can get the orders info
JOIN Person.Address PA --but then joining it with the person addresses
    ON SOH.ShipToAddressID = PA.AddressID --so I can link to the province table
JOIN Person.StateProvince SP --which I do here, so I can get state/province names
    ON PA.StateProvinceID = SP.StateProvinceID --and link by the province ID
JOIN Person.CountryRegion CR --and then add in the country region table so I can get country names
    ON SP.CountryRegionCode = CR.CountryRegionCode --and link on the country code
GROUP BY CR.Name, SP.Name --then group by Country name, then province name
HAVING COUNT(SOH.SalesOrderID) >= 100 --and then I recquire that the COUNT of the Sales Order ID to be 100 or more
ORDER BY CR.Name ASC, COUNT(SOH.SalesOrderID) DESC --and then sort by ascending 

(26 rows affected)

Total execution time: 00:00:00.146

Country_Name,State_Province_Name,Orders
Australia,New South Wales,3009
Australia,Victoria,1710
Australia,Queensland,1461
Australia,South Australia,466
Australia,Tasmania,197
Canada,British Columbia,3472
Canada,Ontario,339
Canada,Quebec,146
France,Seine (Paris),575
France,Seine Saint Denis,391


<span style="font-size: 12pt; font-family: &quot;Times New Roman&quot;, serif; color: rgb(0, 0, 0); background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; font-variant-alternates: normal; vertical-align: baseline; white-space-collapse: preserve;">3. Produce a report showing the total number of units for each product sold by AdventureWorks that also includes the category of each product and each product's sales rank and quintile within the category.&nbsp; Make sure the result set is alphabetical by category and in the appropriate rank order.</span>

In [132]:
WITH Temp_Data AS --For this, I made a CTE so I could then Rank and Ntile them in the second select statement
                    (SELECT PP.Name Product_Name --grabing the product name from Production.Product
                            ,PC.Name AS Product_Category --the product category name
                            ,COUNT(SOD.OrderQty) Number_of_Units_Sold --and count of the units sold
                        FROM Sales.SalesOrderDetail SOD --from the Sales Order Detail so I can get Order Quantity
                        JOIN Sales.SpecialOfferProduct SOP --joined with Special Offer Product
                            ON SOD.ProductID = SOP.ProductID --based on Product ID
                        JOIN Production.ProductProductPhoto PPP --So I can link to Product Photo
                            ON SOD.ProductID = PPP.ProductID --Also on Product ID
                        JOIN Production.Product PP --So I can then get to the product table so I can get the product name
                            ON PPP.ProductID = PP.ProductID --also on product ID
                        JOIN Production.ProductSubcategory PS --and then got to the product subcategory so I can get the subcategory name
                            ON PS.ProductSubcategoryID = PP.ProductSubcategoryID  --linked by the subcategory ID
                        JOIN Production.ProductCategory PC --so I could finally get the category name (That was a lot of joins!)
                            ON PC.ProductCategoryId = PS.ProductCategoryId --based on the category ID
                        GROUP BY PC.Name, PP.Name) --then grouping them by the Category name, then product name
SELECT Product_Category --now I am making a select from my CTE, grabing the product catgeory
       ,Product_Name --the product name
      ,Number_of_Units_Sold, --and number of units sold
RANK() OVER (PARTITION BY Product_Category ORDER BY Number_of_Units_Sold DESC) AS Product_Rank, --Then I ranked the products inside of their category
NTILE(5) OVER (PARTITION BY Product_Category ORDER BY Number_of_Units_Sold DESC) AS Category_Quintile --then added in the quintile inside of the category
FROM Temp_Data --all from the CTE Temp_data
ORDER BY Product_Category ASC, Product_Rank ASC --and then ordered by the Category and Product_Rank

(266 rows affected)

Total execution time: 00:00:00.135

Product_Category,Product_Name,Number_of_Units_Sold,Product_Rank,Category_Quintile
Accessories,Water Bottle - 30 oz.,18752,1,1
Accessories,"Sport-100 Helmet, Blue",15450,2,1
Accessories,"Sport-100 Helmet, Red",15415,3,1
Accessories,"Sport-100 Helmet, Black",15035,4,1
Accessories,Patch Kit/8 Patches,6708,5,1
Accessories,Bike Wash - Dissolver,5308,6,2
Accessories,Hydration Pack - 70 oz.,3222,7,2
Accessories,Hitch Rack - 4-Bike,3184,8,2
Accessories,Mountain Tire Tube,3095,9,2
Accessories,HL Mountain Tire,2792,10,2
